## ACS Housing Units by Building Size for Urban Villages, City of Phoenix

Keep all building size categories as presented by ACS, add a couple of multifamily summary columns (5-20 units, 20+ units) in case needed for dealing with MOEs and statistical reliability

- https://www.census.gov/data/developers/data-sets/acs-5year.html
- https://api.census.gov/data/2013/acs/acs5/variables.html
- https://api.census.gov/data/2021/acs/acs5/variables.html

In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
import getters as get
import utilcalcs as calc
import geo_agg

In [3]:
#Search parameters - NOTE DIFFERENT VARIABLE # ASSIGNMENTS IN 2010 VS 2020
y1 = '2021'
#y0_5 = '2013'
y0_1 = '2010'

cols = 'group(B25004)'
cols_v = 'group(B25005)'

source_1 = 'acs/acs1'
source_5 = 'acs/acs5'

In [4]:
#comp cities and big geo rename dictionary
rename_geos = {'0455000':'Phoenix', '04013':'Maricopa', '0100000US':'US', '0400000US04':'AZ',\
               '0473000':'Tempe','0465000':'Scottsdale', '0427820':'Glendale',\
               '4865000':'San Antonio', '4835000':'Houston', '1235000':'Jacksonville'}

In [12]:
rename_vars = {'B25004_001E':'tot_E','B25004_001M':'tot_M',\
               'B25004_002E':'ForRent_E','B25004_002M':'ForRent_M',\
               'B25004_003E':'RentUnOc_E','B25004_003M':'RentUnOc_M',\
               'B25004_004E':'ForSale_E','B25004_004M':'ForSale_M',\
               'B25004_005E':'Sold_E','B25004_005M':'Sold_M',\
               'B25004_006E':'Seas_E','B25004_006M':'Seas_M',\
               'B25004_007E':'Mig_E','B25004_007M':'Mig_M',\
               'B25004_008E':'Oth_E','B25004_008M':'Oth_M'}

#### Calculate building size columns function

In [13]:
# make city and larger geos
def make_table(source,year,cols,geotype='big'):
    if geotype=='big':
        ph = get.get_phx(source,year,cols)
        mar = get.get_maricopa(source,year,cols)
        us = get.get_us(source,year,cols)
        az = get.get_az(source,year,cols)
        cit = get.get_comp_cities(source,year,cols)
        df = pd.concat([ph,mar,us,az,cit])
        df = df.drop(['NAME','us','state'],axis=1)
        aggtype='GEO_ID'
    elif geotype=='az_places':
        df = get.get_az_plc(source,year,cols)
        aggtype='GEO_ID'
    else:
        pass
    df.rename(rename_vars,axis=1,inplace=True)
    df = get.clean_table(df)
    df = geo_agg.sumgeo_cv(df,aggtype)
    
    for col in df.columns[1:]:
        df.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return df

## 1-year table for for city & larger geos

In [14]:
geo_y1_1y = make_table(source_1,y1,cols)
geo_y0_1y = make_table(source_1,y0_1,cols)

In [18]:
change_vars = ['tot','ForRent','RentUnOc','ForSale',\
               'Sold','Seas','Mig','Oth']

In [19]:
df_1y = pd.merge(geo_y0_1y,geo_y1_1y,how='left',on='GEO_ID')

In [20]:
dff = calc.calc_change(df_1y,y0_1,y1,change_vars)

In [22]:
dff.GEO_ID = dff.GEO_ID.map(rename_geos)

### try 1-year for AZ places

In [29]:
pl_y1_1y = make_table(source_1,y1,cols,'az_places')
pl_y0_1y = make_table(source_1,y0_1,cols,'az_places')

In [30]:
pl_1 = pd.merge(pl_y0_1y,pl_y1_1y,how='left',on='GEO_ID')
pl_1 = calc.calc_change(pl_1,y0_1,y1,change_vars)

## 5-year for AZ places and urban villages

#### other places AZ - 5YR

In [24]:
pl_y1_5y = make_table(source_5,y1,cols,'az_places')
pl_y0_5y = make_table(source_5,y0_1,cols,'az_places')

In [26]:
pl = pd.merge(pl_y0_5y,pl_y1_5y,how='left',on='GEO_ID')
pl = calc.calc_change(pl,y0_1,y1,change_vars)

In [27]:
pl.head()

,GEO_ID,ForSale_10E,ForSale_10M,ForSale_10C,Sold_10E,Sold_10M,Sold_10C,Seas_10E,Seas_10M,Seas_10C,...,Mig_1021E,Mig_1021M,Mig_1021C,Mig_10P,Mig_21P,Oth_1021E,Oth_1021M,Oth_1021C,Oth_10P,Oth_21P
0,Aguila,0.0,132.0,0.000000,0.0,132.0,0.000000,9.0,14.0,94.562648,...,0.0,132.638607,NaN,0.0,0.0,-16.0,32.649655,124.048843,0.640000,0.470588
1,Anthem,517.0,204.0,23.986878,189.0,132.0,42.456699,1232.0,246.0,12.138318,...,0.0,134.346567,NaN,0.0,0.0,35.0,105.152271,182.635294,0.032619,0.103960
2,Arlington,0.0,132.0,0.000000,0.0,132.0,0.000000,4.0,7.0,106.382979,...,0.0,132.638607,NaN,0.0,NaN,0.0,132.638607,NaN,0.000000,NaN
3,Avondale,803.0,305.0,23.089705,172.0,110.0,38.877501,245.0,164.0,40.692265,...,0.0,135.591298,NaN,0.0,0.0,-354.0,274.798108,47.189413,0.230636,0.221986
4,Buckeye,503.0,161.0,19.457722,209.0,121.0,35.194369,499.0,189.0,23.024773,...,0.0,135.591298,NaN,0.0,0.0,-402.0,314.268993,47.523627,0.320763,0.105871


## Occupancy Status

In [35]:
df_y1_1y = make_table(source_1,y1,cols_v)
df_y0_1y = make_table(source_1,y0_1,cols_v)
dff_y1_1y = make_table(source_1,y1,cols_v,'az_places')
dff_y0_1y = make_table(source_1,y0_1,cols_v,'az_places')

In [36]:
df_y1_1y.head()

,GEO_ID,B25005_00321E,B25005_00321M,B25005_00321C,B25005_00221E,B25005_00221M,B25005_00221C,B25005_00121E,B25005_00121M,B25005_00121C
0,0455000,39936.0,3305.0,5.030846,2650.0,1234.0,28.307622,42586.0,3312.0,4.727783
1,04013,130713.0,7109.0,3.306160,10356.0,2179.0,12.790846,141069.0,7183.0,3.095340
2,0100000US,14081560.0,95249.0,0.411191,521760.0,15543.0,1.810915,14603320.0,99060.0,0.412364
3,0400000US04,299075.0,10334.0,2.100499,21887.0,3273.0,9.090628,320962.0,10859.0,2.056697
4,4865000,53386.0,4582.0,5.217492,1039.0,930.0,54.412853,54425.0,4667.0,5.212829


In [37]:
df_y0_1y.head()

,GEO_ID,B25005_00310E,B25005_00310M,B25005_00310C,B25005_00210E,B25005_00210M,B25005_00210C,B25005_00110E,B25005_00110M,B25005_00110C
0,0455000,87048.0,5297.0,3.699178,2890.0,942.0,19.814684,89938.0,5280.0,3.568821
1,04013,238006.0,9486.0,2.422866,18084.0,2358.0,7.926535,256090.0,9832.0,2.333906
2,0100000US,16290352.0,166463.0,0.621186,933294.0,16788.0,1.093489,17223646.0,167247.0,0.590293
3,0400000US04,476093.0,12794.0,1.633611,36595.0,3454.0,5.737658,512688.0,12983.0,1.539416
4,4865000,53772.0,4456.0,5.037593,1804.0,682.0,22.981689,55576.0,4469.0,4.888292


## Combine 1yr and 5 yr

In [34]:
with pd.ExcelWriter(f'output/acs_vacancy_status.xlsx') as writer:
    dff.to_excel(writer, sheet_name="1yr_big_geos", index=False)
    pl_1.to_excel(writer, sheet_name="1yr_places", index=False)
    pl.to_excel(writer, sheet_name="5yr_places", index=False)